In [2]:
!pip install pyshark

# out_00000_20210719001654

In [5]:
import pyshark
import asyncio

# Define an async function to capture packets
async def read_pcap_async(pcap_file):
    cap = pyshark.FileCapture(pcap_file)
    
    # Iterate through packets asynchronously
    for packet in cap:
        print(packet)  # You can access packet details here

    # Close capture when done
    await cap.close()

# Run the asynchronous function using asyncio's run method
pcap_file = 'out_00000_20210719001654.pcap'
await read_pcap_async(pcap_file)


RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
from scapy.all import rdpcap, Raw
from scapy.layers.inet import IP, UDP
import struct
def parse_arcabook_packet(data):
    msg_size = struct.unpack('<H', data[0:2])[0]
    msg_type = struct.unpack('<H', data[2:4])[0]
    
    parsed_data = {
        'Msg Size': msg_size,
        'Msg Type': msg_type
    }
    
    if msg_type == 100:  # Add Order Message
        parsed_data.update({
            'SourceTimeNS': struct.unpack('<I', data[4:8])[0],
            'SymbolIndex': struct.unpack('<I', data[8:12])[0],
            'SymbolSeqNum': struct.unpack('<I', data[12:16])[0],
            'OrderID': struct.unpack('<I', data[16:20])[0],
            'Price': struct.unpack('<I', data[20:24])[0],
            'Volume': struct.unpack('<I', data[24:28])[0],
            'Side': data[28:29].decode('ascii'),
            'OrderIDGTCIndicator': data[29],
            'TradeSession': data[30]
        })
    elif msg_type == 101:  # Modify Message
        parsed_data.update({
            'SourceTimeNS': struct.unpack('<I', data[4:8])[0],
            'SymbolIndex': struct.unpack('<I', data[8:12])[0],
            'SymbolSeqNum': struct.unpack('<I', data[12:16])[0],
            'OrderID': struct.unpack('<I', data[16:20])[0],
            'Price': struct.unpack('<I', data[20:24])[0],
            'Volume': struct.unpack('<I', data[24:28])[0],
            'Side': data[28:29].decode('ascii'),
            'OrderIDGTCIndicator': data[29],
            'ReasonCode': data[30]
        })
    elif msg_type == 102:  # Delete Message
        parsed_data.update({
            'SourceTimeNS': struct.unpack('<I', data[4:8])[0],
            'SymbolIndex': struct.unpack('<I', data[8:12])[0],
            'SymbolSeqNum': struct.unpack('<I', data[12:16])[0],
            'OrderID': struct.unpack('<I', data[16:20])[0],
            'Side': data[20:21].decode('ascii'),
            'OrderIDGTCIndicator': data[21],
            'ReasonCode': data[22]
        })
    elif msg_type == 103:  # Execution Message
        parsed_data.update({
            'SourceTimeNS': struct.unpack('<I', data[4:8])[0],
            'SymbolIndex': struct.unpack('<I', data[8:12])[0],
            'SymbolSeqNum': struct.unpack('<I', data[12:16])[0],
            'OrderID': struct.unpack('<I', data[16:20])[0],
            'Price': struct.unpack('<I', data[20:24])[0],
            'Volume': struct.unpack('<I', data[24:28])[0],
            'OrderIDGTCIndicator': data[28],
            'ReasonCode': data[29],
            'TradeID': struct.unpack('<I', data[30:34])[0]
        })
    # Add more message types as needed
    
    return parsed_data

# Specify the path to your PCAP file


pcap_directory = 'C:\\Users\\nmasc\\Documents\\para\\projects\\QuantEdge\\marketmaking\\NYSEBQT_CVOLCTS_PUB_A_07_19_2021_mid_8pm'
import os

# Loop over each file in the directory
for filename in os.listdir(pcap_directory):
    if filename.endswith('.pcap'):
        pcap_file = filename

        # Read the PCAP file
        packets = rdpcap(pcap_file)
        # Iterate through the packets and print details
        for packet in packets:
            # If the packet contains a UDP layer, extract specific details
            
            if UDP in packet:
                # Access IP layer
                ip_layer = packet[IP]
                src_ip = ip_layer.src
                dst_ip = ip_layer.dst

                # Access UDP layer
                udp_layer = packet[UDP]
                src_port = udp_layer.sport
                dst_port = udp_layer.dport


                # If there's a Raw layer, convert the payload to decimal
                if Raw in packet:
                    raw_data = packet[Raw].load  # Extract raw payload
                    decimal_values = [byte for byte in raw_data]  # Convert bytes to decimal
                    if decimal_values[2] != 1 and int(decimal_values[2]) < 200:
                        parsed_packet = parse_arcabook_packet(raw_data)
                        if parsed_packet['Msg Type'] != 267 and parsed_packet['Msg Type'] != 268:
                            print(parsed_packet)


In [5]:
from scapy.all import *
import struct

def process_pcap(pcap_file):
    packets = rdpcap(pcap_file)
    
    for packet in packets:
        if "UDP" in packet.summary():
            continue
        else:
            print(packet.summary())

# Replace 'your_file.pcap' with the path to your PCAP file
pcap_directory = 'C:\\Users\\nmasc\\Documents\\para\\projects\\QuantEdge\\marketmaking\\NYSEBQT_CVOLCTS_PUB_A_07_19_2021_mid_8pm'
import os

# Loop over each file in the directory
for filename in os.listdir(pcap_directory):
    if filename.endswith('.pcap'):
        pcap_file = filename
        process_pcap(pcap_file)

In [1]:
from scapy.all import *
import csv

# Define message types and field mappings based on spec
message_types = {
    '100': 'Add Order',
    '101': 'Modify Order',
    '102': 'Delete Order',
    '103': 'Execution',
    '105': 'Imbalance',
    '106': 'Add Order Refresh',
    '107': 'Attributed Add Order', 
    '108': 'Attributed Add Order Refresh'
}

field_mappings = {
    '100': ['MsgSize', 'MsgType', 'SourceTimeNS', 'SymbolIndex', 'SymbolSeqNum', 'OrderID', 'Price', 'Volume', 'Side', 'OrderIDGTCIndicator', 'TradeSession'],
    # Define mappings for other message types
}

# Read PCAP file
packets = rdpcap("onepacket.pcap")

# Open CSV file for writing
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(['Timestamp', 'MessageType'] + field_mappings['100'])  # Using '100' fields as example
    
    # Process each packet
    for packet in packets:
        if TCP in packet and packet[TCP].payload:
            data = bytes(packet[TCP].payload)
            
            # Extract message type
            msg_type = data[2:4].decode()
            
            if msg_type in message_types:
                # Extract fields based on message type
                fields = field_mappings[msg_type]
                values = [packet.time, message_types[msg_type]]
                
                for field in fields:
                    # Extract value for each field
                    # This would require implementing logic for each field type
                    values.append(extract_field(data, field))
                
                # Write row to CSV
                writer.writerow(values)

def extract_field(data, field):
    # Implement extraction logic for each field type
    # This would be based on the field definitions in the spec
    pass